# Prelude

In [1]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CONF_FLAG = ["randconfig", "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]
SOLU_SIZE = 1000 # Initial number of solutions to extract

# Loading data

In [2]:
#
#
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name
            
def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

# Build a dict {option: val} from .config file
def get_dict(filename):
    res = dict()
    with open(filename, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

# Extracting solutions

In [3]:
# Extract the SOLU_SIZE solutions
fla = CNF(DIMACS_FLA)
models = []
with Solver(bootstrap_with=fla.clauses) as s:
    s.solve()
    for no, k in enumerate(s.enum_models(), 1):
        if no > SOLU_SIZE:
            break
        models.append(k)

In [4]:
# Erase every variables that are not "real" features
config_models = list(map(lambda x: {variables[k] for k in x if k > 0 and k in variables}, models))

In [5]:
# Get rid of redundant solutions
lon = len(config_models)
res = list()
current = 0
in_list = True
while(in_list):
    res.append(config_models[current])
    for k in range(current + 1, lon):
        if config_models[current] != config_models[k]:
            current = k
            break
        if k == lon - 1:
            in_list = False

In [6]:
potential_configurations = res
len(potential_configurations)

108

# Config Generation

In [7]:
# Dictionary {option : [possible_values+]}
feature = dict()
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c") and '=' in line:
            m = re.search('(c) (\d+) (\w+)=([\w"-/]*)', line)
#             print(m.group(3), m.group(4))
            if m.group(4) != 'nonempty' and m.group(4) != 'n':
                if m.group(3) in feature:
                    feature[m.group(3)].append(m.group(4))
                else:
                    feature[m.group(3)] = [m.group(4)]

In [8]:
import random

# Write .config content using the solution and 
# pick a random value in the list as real value in
# the configuration
def create_file_content(res):
    df_options = pd.read_csv('examples/alloptions-x86.4.14.152.csv')
    # file content
    file = ""
    for elt in res:
        if "CHOICE_" in elt:
            continue
        kconfig_type = search_kconfig_type(elt, df_options)
        if kconfig_type == 'BOOL':
    #         print("CONFIG_{}=y".format(elt))
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == 'TRISTATE':
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == "STRING":
            if elt == "IMA_DEFAULT_HASH":
                found = False
                for k in res:
                    if "IMA_DEFAULT_HASH_" in k and k != "IMA_DEFAULT_HASH":
                        file += 'CONFIG_{}="{}"\n'.format(elt, k[17:].lower())
                        found = True
                if not found:
                    file += 'CONFIG_{}=""\n'.format(elt)
            else:
                file += 'CONFIG_{}="{}"\n'.format(elt, random.choice(feature[elt]))
        elif kconfig_type == "INT":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
        elif kconfig_type == "HEX":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
    return file

In [9]:
# Create .config (experimentation)
def create_config_xp(res):
    file_content = create_file_content(res)
    with open('../linux-4.14.152/.config', 'w') as stream:
        stream.write("# EXPERIMENTATION\n##################\n")
        stream.write(file_content)

In [10]:
# potential_configurations = potential_configurations[:3]


In [16]:
diff_opt_occ = dict()
for solu in potential_configurations:
    create_config_xp(solu)
    os.system("make olddefconfig -C {}".format(CONFIG_DIR))
    new_config = get_dict("{}/.config".format(CONFIG_DIR))
    # diff
    if len(new_config) > len(solu):
        diff = set(new_config) - solu
        for feat in diff:
            if feat in diff_opt_occ:
                diff_opt_occ[feat] += 1
            else:
                diff_opt_occ[feat] = 1
    

In [15]:
# Saving data
file_content = "option;occurence\n"
for option in diff_opt_occ.keys():
    file_content += "{};{}\n".format(option, diff_opt_occ[option])
with open("examples/diff_occ.csv", "w") as stream:
    stream.write(file_content)

In [10]:
diff_opt_occ

NameError: name 'diff_opt_occ' is not defined

In [18]:
for k in diff_opt_occ.keys():
    if diff_opt_occ[k] != 108:
        print(k)

BOUNCE
CAN_CALC_BITTIMING
SQUASHFS_ZLIB
SCSI_SAS_HOST_SMP
OCFS2_FS_STATS
CAPI_TRACE
XEN_SCRUB_PAGES
COMMON_CLK_XGENE
MSDOS_PARTITION
FB_ATY_BACKLIGHT


In [27]:
len(diff_opt_occ)

414

# Playground

In [11]:
# Read file of impossible values
def read_imp_val_csv(file):
    df = pd.read_csv(file)
    df_dict = df.to_dict()
    res = dict()
    for k, v in df_dict['option'].items():
        imp = map(lambda x: x.replace("'", ''), 
                  df_dict['impossible_value'][k][1:-1].split()) 
        res[v] = set(imp)
    return res
impossible_values = read_imp_val_csv('examples/impossible.csv')

In [12]:
# Read file of diff and occurences
df = pd.read_csv("examples/diff_occ.csv", sep=";")
diff_opt_occ = df.set_index('option')['occurence'].to_dict()

In [13]:
diff_opt_occ

{'USB_PCI': 108,
 'IR_MCE_KBD_DECODER': 108,
 'TRACEPOINTS': 108,
 'HW_RANDOM_MTK': 108,
 'FRAMEBUFFER_CONSOLE_DETECT_PRIMARY': 108,
 'WLAN_VENDOR_TI': 108,
 'SND_VERBOSE_PROCFS': 108,
 'HID_ITE': 108,
 'LDISC_AUTOLOAD': 108,
 'PREVENT_FIRMWARE_BUILD': 108,
 'AIC79XX_REG_PRETTY_PRINT': 108,
 'RT2800PCI_RT53XX': 108,
 'X86_MCE_THRESHOLD': 108,
 'DVB_NET': 108,
 'CAN_BCM': 108,
 'IR_XMP_DECODER': 108,
 'SCSI_PROC_FS': 108,
 'RSI_USB': 108,
 'HID_PICOLCD_LCD': 108,
 'SECTION_MISMATCH_WARN_ONLY': 108,
 'FB_RADEON_I2C': 108,
 'ACPI_SLEEP': 108,
 'HW_RANDOM': 108,
 'KVM_GUEST': 108,
 'VIDEO_GO7007_LOADER': 108,
 'COMPAT_BRK': 108,
 'HID_BELKIN': 108,
 'NET_VENDOR_VIA': 108,
 'RT2800USB_RT33XX': 108,
 'NET_VENDOR_SYNOPSYS': 108,
 'IDEPCI_PCIBUS_ORDER': 108,
 'SH_DMAE_BASE': 108,
 'LOGO_LINUX_CLUT224': 108,
 'UBIFS_FS_SECURITY': 108,
 'CRYPTO_LZO': 108,
 'CPU_IDLE': 108,
 'HID_KENSINGTON': 108,
 'HID_A4TECH': 108,
 'INET6_XFRM_MODE_TRANSPORT': 108,
 'CONTEXT_TRACKING_FORCE': 108,
 'HISI_THERMA

In [34]:
# count core features from diff dict
core_f = 0
hasnt_iv = 0
for opt in diff_opt_occ:
    if opt in impossible_values :
        if 'n' in impossible_values[opt]:
            core_f += 1
    else:
        hasnt_iv += 1
print("{}/{} have not impossible values\n{}/{} of core features".format(hasnt_iv, len(diff_opt_occ),
                                                                        core_f, len(diff_opt_occ)))

414/414 have not impossible values
0/414 of core features


In [35]:
len(config_models[0])

2182

In [26]:
nb = 0
for k in diff_opt_occ.keys():
    if diff_opt_occ[k] == 108:
        nb += 1

In [27]:
nb

404

In [28]:
diff_opt_occ['THERMAL_HWMON'] == diff_opt_occ['THERMAL']

True

In [34]:
diff_opt_occ2 = dict()
for solu in potential_configurations[:4]:
    create_config_xp(solu)
    os.system("make oldconfig -C {}".format(CONFIG_DIR))
    new_config = get_dict("{}/.config".format(CONFIG_DIR))
    # diff
    if len(new_config) > len(solu):
        diff = set(new_config) - solu
        for feat in diff:
            if feat in diff_opt_occ2:
                diff_opt_occ2[feat] += 1
            else:
                diff_opt_occ2[feat] = 1

In [35]:
diff_opt_occ2

{'USB_PCI': 4,
 'IR_MCE_KBD_DECODER': 4,
 'TRACEPOINTS': 4,
 'HW_RANDOM_MTK': 4,
 'FRAMEBUFFER_CONSOLE_DETECT_PRIMARY': 4,
 'WLAN_VENDOR_TI': 4,
 'SND_VERBOSE_PROCFS': 4,
 'HID_ITE': 4,
 'LDISC_AUTOLOAD': 4,
 'PREVENT_FIRMWARE_BUILD': 4,
 'AIC79XX_REG_PRETTY_PRINT': 4,
 'RT2800PCI_RT53XX': 4,
 'X86_MCE_THRESHOLD': 4,
 'DVB_NET': 4,
 'CAN_BCM': 4,
 'IR_XMP_DECODER': 4,
 'SCSI_PROC_FS': 4,
 'RSI_USB': 4,
 'HID_PICOLCD_LCD': 4,
 'SECTION_MISMATCH_WARN_ONLY': 4,
 'FB_RADEON_I2C': 4,
 'ACPI_SLEEP': 4,
 'HW_RANDOM': 4,
 'KVM_GUEST': 4,
 'VIDEO_GO7007_LOADER': 4,
 'COMPAT_BRK': 4,
 'HID_BELKIN': 4,
 'NET_VENDOR_VIA': 4,
 'RT2800USB_RT33XX': 4,
 'NET_VENDOR_SYNOPSYS': 4,
 'IDEPCI_PCIBUS_ORDER': 4,
 'SH_DMAE_BASE': 4,
 'LOGO_LINUX_CLUT224': 4,
 'UBIFS_FS_SECURITY': 4,
 'CRYPTO_LZO': 4,
 'CPU_IDLE': 4,
 'HID_KENSINGTON': 4,
 'HID_A4TECH': 4,
 'INET6_XFRM_MODE_TRANSPORT': 4,
 'CONTEXT_TRACKING_FORCE': 4,
 'HISI_THERMAL': 4,
 'MOUSE_PS2_ALPS': 4,
 'HVC_XEN_FRONTEND': 4,
 'USB_BELKIN': 4,
 'IGB_DCA

In [49]:
test = dict()
solu = potential_configurations[10]
# ajout de THERMAL
solu.add("THERMAL_HWMON")
# Création de la config
create_config_xp(solu)
# Différence
os.system("make olddefconfig -C {}".format(CONFIG_DIR))
# new config file
new_config = get_dict("{}/.config".format(CONFIG_DIR))
# diff
diff = set(new_config) - solu
for feat in diff:
    if feat in test:
        test[feat] += 1
    else:
        test[feat] = 1

In [50]:
test

{'USB_PCI': 1,
 'IR_MCE_KBD_DECODER': 1,
 'TRACEPOINTS': 1,
 'HW_RANDOM_MTK': 1,
 'FRAMEBUFFER_CONSOLE_DETECT_PRIMARY': 1,
 'WLAN_VENDOR_TI': 1,
 'SND_VERBOSE_PROCFS': 1,
 'HID_ITE': 1,
 'LDISC_AUTOLOAD': 1,
 'PREVENT_FIRMWARE_BUILD': 1,
 'AIC79XX_REG_PRETTY_PRINT': 1,
 'RT2800PCI_RT53XX': 1,
 'X86_MCE_THRESHOLD': 1,
 'DVB_NET': 1,
 'CAN_BCM': 1,
 'IR_XMP_DECODER': 1,
 'SCSI_PROC_FS': 1,
 'RSI_USB': 1,
 'HID_PICOLCD_LCD': 1,
 'SECTION_MISMATCH_WARN_ONLY': 1,
 'FB_RADEON_I2C': 1,
 'ACPI_SLEEP': 1,
 'HW_RANDOM': 1,
 'KVM_GUEST': 1,
 'VIDEO_GO7007_LOADER': 1,
 'COMPAT_BRK': 1,
 'HID_BELKIN': 1,
 'NET_VENDOR_VIA': 1,
 'RT2800USB_RT33XX': 1,
 'NET_VENDOR_SYNOPSYS': 1,
 'IDEPCI_PCIBUS_ORDER': 1,
 'SH_DMAE_BASE': 1,
 'LOGO_LINUX_CLUT224': 1,
 'UBIFS_FS_SECURITY': 1,
 'CRYPTO_LZO': 1,
 'CPU_IDLE': 1,
 'HID_KENSINGTON': 1,
 'HID_A4TECH': 1,
 'INET6_XFRM_MODE_TRANSPORT': 1,
 'CONTEXT_TRACKING_FORCE': 1,
 'HISI_THERMAL': 1,
 'MOUSE_PS2_ALPS': 1,
 'HVC_XEN_FRONTEND': 1,
 'USB_BELKIN': 1,
 'IGB_DCA

In [51]:
test['THERMAL']

KeyError: 'THERMAL'

In [40]:
solu = potential_configurations[100]
create_config_xp(solu)


In [44]:
def get_var(option):
    for k, v in variables.items():
        if v == option:
            return k

In [45]:
get_var("THERMAL")

3303

In [85]:
fla = CNF(DIMACS_FLA)
SOLU_SIZE = 1
m = []
with Solver(bootstrap_with=fla.clauses) as s:
    for no, k in enumerate(s.enum_models(assumptions=[-get_var("THERMAL")]), 1):
        if no > SOLU_SIZE:
            break
        m.append(k)

In [95]:
s1 = list()
with Solver(bootstrap_with=fla.clauses) as s:
    for no, k in enumerate(s.enum_models(assumptions=[-get_var("THERMAL")]), 1):
        if no > 1:
            break
        s1.append(k)

In [98]:
s1 = list(map(lambda x: {variables[k] for k in x if k > 0 and k in variables}, s1))

In [101]:
create_config_xp(s1[0])

In [86]:
len(m[0])

76662

In [87]:
m = list(map(lambda x: {variables[k] for k in x if k > 0 and k in variables}, m))

In [73]:
# Get rid of redundant solutions
lon = len(m)
res = list()
current = 0
in_list = True
while(in_list):
    res.append(m[current])
    for k in range(current + 1, lon):
        if m[current] != m[k]:
            current = k
            break
        if k == lon - 1:
            in_list = False

KeyboardInterrupt: 

In [90]:
len(m[0])

2193

In [102]:
create_config_xp(m[0])

In [79]:
s = set()
for k in res:
    s.add("THERMAL" in k)
s

{False}

In [ ]:
diff_test = dict()
for solu in res:
    create_config_xp(solu)
    os.system("make olddefconfig -C {}".format(CONFIG_DIR))
    new_config = get_dict("{}/.config".format(CONFIG_DIR))
    # diff
    if len(new_config) > len(solu):
        diff = set(new_config) - solu
        for feat in diff:
            if feat in diff_test:
                diff_test[feat] += 1
            else:
                diff_test[feat] = 1
    

In [63]:
for elt in m[0]:
    if 'THERMAL' in elt:
        print(elt)


THERMAL
THERMAL_GOV_STEP_WISE
THERMAL_EMERGENCY_POWEROFF_DELAY_MS
HISI_THERMAL
THERMAL_DEFAULT_GOV_STEP_WISE


In [64]:
for elt in potential_configurations[0]:
    if 'THERMAL' in elt:
        print(elt)


THERMAL_EMERGENCY_POWEROFF_DELAY_MS


In [66]:
create_config_xp(m[0])


In [24]:
create_config_xp(potential_configurations[0])